In [47]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import numpy as np

%pylab inline
pylab.rcParams['figure.figsize'] = (15,9)

Populating the interactive namespace from numpy and matplotlib


In [48]:
def aggregate(df):
    return df.groupby(0)[df.columns[1:54]].min()

In [ ]:
out_dir = '../papi_output/'
path = out_dir + 'o3m.csv'

df = pd.read_csv(path, header=None)
df = aggregate(df)
df = df.sample(frac=1) # shuffle for better cross-validation

In [50]:
x = df.iloc[:,0:52].as_matrix()
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
y = df.iloc[:,52].as_matrix()

In [ ]:
pca = PCA(n_components=6)
pca.fit(x)
x2 = pca.transform(x)

In [ ]:
svr = SVR(kernel='linear')
svr.fit(x2, y)

In [ ]:
scores = cross_val_score(svr, x2, y, cv=3)
scores.mean()

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
# just to visualize which of 52 PAPI output events are most significant (marked in yellow)

fig, axs = plt.subplots(nrows=6)
for i, ax in enumerate(axs): 
    ax.imshow(pca.components_[i:i+1])
plt.show()